In [ ]:
import numpy as np
import pandas as pd
import requests
import time
import os

from bs4 import BeautifulSoup
from urllib.parse import quote
from selenium import webdriver
from collections import Counter

In [ ]:
# /Users/hyeonju/data-crawling/chromedriver
driver = webdriver.Chrome(executable_path = '/Users/hyeonju/data-crawling/chromedriver')

In [ ]:
# url 설정
url = f'http://ticket.interpark.com/TPGoodsList.asp?Ca=Eve&SubCa=Eve_O&tid4=Eve_O&smid1=gnb&smid2=008&smid3=1'

# requests로 데이터 요청하기
resp = requests.get(url)

# 설정한 url 로 데이터 get 함수 요청
driver.get(url)

In [ ]:
# # 화면상에서 스크롤 끝까지 내리기
# SCROLL_PAUSE_SEC = 1

# # 스크롤 높이 가져옴
# last_height = driver.execute_script("return document.body.scrollHeight")

# while True:
#     # 끝까지 스크롤 다운
#     driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

#     # 1초 대기
#     time.sleep(SCROLL_PAUSE_SEC)

#     # 스크롤 다운 후 스크롤 높이 다시 가져옴
#     new_height = driver.execute_script("return document.body.scrollHeight")
#     if new_height == last_height:
#         break
#     last_height = new_height

In [ ]:
# 접근한 페이지의 소스코드를 텍스트로 저장 (Str)
page_text = driver.page_source

# page_text를 html list로 변환 
html = BeautifulSoup(page_text, 'html.parser')

In [ ]:
# body > table > tbody > tr:nth-child(2) > td:nth-child(3) > div > div > div.con > div > table > tbody > tr:nth-child(1) > td.RKtxt > span > a

# blog_url
url=[]

for i in html.select('td.RKtxt > span > a') :
    urls = i.attrs['href']
    url.append(urls)
    
print(f"전시회{len(url)}개 의 주소입니다.")

print(url)

In [ ]:
goods_num_list = []

for i in url :
    goods_num = i.split('=')[1]
    goods_num_list.append(goods_num)
    
print(f"전시회 {len(url)} 개의 goods_num_list 입니다.")

print(goods_num_list)

In [ ]:
exhibition_url = []

for i in goods_num_list :
    exhibition_url.append(f'http://ticket.interpark.com/Ticket/Goods/GoodsInfo.asp?GoodsCode={i}')
    
print(f"전시회 {len(exhibition_url)} 개의 exhibition_url 입니다.")

print(exhibition_url)

# 전시회 세부페이지

In [ ]:

# 이후 상세페이지 조회시 for문으로 접근하여 데이터 수집
# 우선 한개만

detail_url_1 = 'http://ticket.interpark.com/Ticket/Goods/GoodsInfo.asp?GoodsCode=22015433'

request = requests.get(detail_url_1)

driver.get(detail_url_1)


In [ ]:
try:
    driver.find_element_by_xpath('//*[@id="popup-prdGuide"]/div/div[3]/div/a').click()
except:
    pass

In [ ]:
# 접근한 페이지의 소스코드를 텍스트로 저장 (Str)
page_text = driver.page_source

# page_text를 html list로 변환 
html = BeautifulSoup(page_text, 'html.parser')

In [ ]:
# title

title_1 = html.select_one('div.summaryTop > h2').text
print(title_1)

In [ ]:
# Place
#container > div.contents > div.productWrapper > div.productMain > div.productMainTop > div > div.summaryBody > ul > li:nth-child(1) > div > a

place_1 = html.select_one('li:nth-child(1) > div > a').text
place_1 = place_1.split('(')[0]
print(place_1)

In [ ]:
# date
#container > div.contents > div.productWrapper > div.productMain > div.productMainTop > div > div.summaryBody > ul > li:nth-child(2) > div > p

date_1 = html.select_one('li:nth-child(2) > div > p').text
print(f'전체 기간: {date_1}')

startAt = date_1.split(' ~')[0]
print(f'전시 시작일 : {startAt}')

endAt = date_1.split(' ~')[1]
print(f'전시 종료일 : {endAt}')


In [ ]:
# ageLimit
#container > div.contents > div.productWrapper > div.productMain > div.productMainTop > div > div.summaryBody > ul > li:nth-child(4) > div > p

ageLimit_1 = html.select_one('li:nth-child(4) > div > p').text
print(f'관람 가능 연령 : {ageLimit_1}')


In [ ]:
# price 
#container > div.contents > div.productWrapper > div.productMain > div.productMainTop > div > div.summaryBody > ul > li.infoItem.infoPrice > div > ul > li:nth-child(2) > span.price

price_1 = html.select_one('li:nth-child(2) > span.price').text

print(f'가격 : {price_1}')

In [ ]:
# main_img
#container > div.contents > div.productWrapper > div.productMain > div.productMainTop > div > div.summaryBody > div > div.posterBoxTop > img

main_img_1 = html.select_one('div.posterBoxTop > img').attrs['src']
main_img_1 = 'http:' + main_img_1

main_img_1

# LHJ 작성

In [101]:
# test_url = 'http://ticket.interpark.com/Ticket/Goods/GoodsInfo.asp?GoodsCode=22015433' # 합스부르크
# test_url = 'http://ticket.interpark.com/Ticket/Goods/GoodsInfo.asp?GoodsCode=22005831' #에바 알머슨
test_url = 'https://tickets.interpark.com/goods/22012710'
# test_url = 'https://tickets.interpark.com/goods/22014814' # 외규장각 의궤

request = requests.get(test_url)

driver.get(test_url)

driver.implicitly_wait(3)

# //*[@id="popup-prdGuide"]/div/div[3]/div/a
# 팝업페이지가 뜬다면 '오늘 하루동안 닫기' 버튼 클릭
# error: 팝업창 뜰 때는 닫아준다. 팝업창 닫은 다음에 크롤링하면 객체가 없어서 찾지 못한다. 

try:
    driver.find_element_by_xpath('//*[@id="popup-prdGuide"]/div/div[3]/div/a').click()
except:
    pass

# 접근한 페이지의 소스코드를 텍스트로 저장
page_text = driver.page_source

# page_text를 html 리스트로 파싱
exhibition_html = BeautifulSoup(page_text, 'html.parser')

In [102]:
# 공연시간 정보 text 추출
# 공통 path: productMainBody > div > div:nth-child(1) > div.class="contentDetail”
# 개별 path
#   p.contentDetailText
#   ul.contentDetailList

# 📌 ISSUE
# [ ] 추출 데이터 정제하기
# [ ] empty element 문제 고민 필요

texts = []
block_1 = exhibition_html.select_one('div:nth-child(1) > div.contentDetail > p').text
block_2 = exhibition_html.select_one('div:nth-child(1) > div > ul > div').text
texts.append(block_1)
texts.append(block_2)
print(texts)



['예매가능시간: 관람 5분 전까지', '* 오픈일\u3000ㆍ수-일요일: 10:00-19:00 (입장시간 기준)* 휴관일ㆍ월·화요일\xa0ㆍ공휴일\u3000\u3000- 1월 1일(일) 신정\u3000\u3000- 1월 22일(일)-1월 24일(화) 설연휴* 소요시간: 100분(1시간 40분)* 관람인원: 1회차 1명-8명* 대표번호: 1877-6145']


In [103]:
# (공지사항 case 1) img 추출
# productMainBody > div > div:nth-child(2) > div.contentDetail
# 📌 ISSUE
# [ ] 데이터 정제하기
# [ ] empty element 제거

notice_image_list = exhibition_html.select('div:nth-child(2) > div.contentDetail > img')
images = []

for i in notice_image_list:
  i = 'http:' + i.get('src') #type: ignore
  images.append(i)

print(images)

[]


In [104]:
# (공지사항 case 2) text 추출
# productMainBody > div > div:nth-child(2) > div.contentDetail
# 📌 ISSUE
# [ ] 데이터 정제하기
# [ ] empty element 제거

notice_text_list = exhibition_html.select_one('div:nth-child(2) > div.contentDetail').text
print(notice_text_list)

# texts = []
# for i in notice_text_list:
#   texts.append(i)
# print(texts)

* 중도입장불가　예매한 시간 15분 전까지 도착해주시기 바랍니다. * 사전 예매 및 문의 대상　ㆍ단체 20인 이상　ㆍ스페셜 이벤트, 워크샵 투어　ㆍ장애인, 임산부, 노약자, 외국인* 관람연령안내　입장 가능 연령은 8세이상 70세이하입니다.　만14세 미만 청소년은 법정 대리인 동의서가 필수입니다.　(페이지 하단부 개인정보 처리 방침 동의서를 확인 바랍니다.)


In [ ]:
# 상세정보 이미지 추출 (완료)
# detail image url list
# productMainBody > div > div.content.description > div.contentDetail

image_url_list = exhibition_html.select('div.content.description > div.contentDetail > img')
images = []

for i in image_url_list:
  i = 'http:' + i.get('src') #type: ignore
  images.append(i)

print(images)

In [ ]:
# 예매자 통계 (재근님 크롤링중)
# 1. 성별 통계
# productMainBody > div > div.content.prdStat > div > div.statGender
# - 컬럼: (남) #productMainBody > div > div.content.prdStat > div > div.statGender > div.statGenderType.is-male > div.statGenderName
# - 컬럼: (여) #productMainBody > div > div.content.prdStat > div > div.statGender > div.statGenderType.is-female > div.statGenderName
# -- 데이터: (남) #productMainBody > div > div.content.prdStat > div > div.statGender > div.statGenderType.is-male > div.statGenderValue
# -- 데이터: (여) #productMainBody > div > div.content.prdStat > div > div.statGender > div.statGenderType.is-female > div.statGenderValue

# gender_stat_list = []

# gender_index1 = exhibition_html.find_all('div.statGenderType.is-male > div.statGenderName').text
# gender_stat_list.append(gender_index1)

# for i in exhibition_html.select('div.content.prdStat > div > div.statGender'):
#   gender_stat_list.append(i.text)

# print(gender_stat_list)

# 2. 연령 통계
# productMainBody > div > div.content.prdStat > div > div.statAge
# 연령별 수치: #productMainBody > div > div.content.prdStat > div > div.statAge > div > div:nth-child(1) > div.statAgePercent
# 연령 범주: #productMainBody > div > div.content.prdStat > div > div.statAge > div > div:nth-child(1) > div.statAgeName

---

### 자동화

### 세부 메서드

In [ ]:
def get_exhibition_info(url) :
        resp = requests.get(url)
        driver.get (url)
        driver.find_element_by_xpath('//*[@id="popup-prdGuide"]/div/div[3]/div/a').click()
        page_text = driver.page_source
        html = BeautifulSoup(resp.text, 'html.parser')
        return html, resp

In [ ]:
def get_exhibition_title(html) :
    title = html.select_one('div.summaryTop > h2').text
    return title

In [ ]:
def get_exhibition_place(html) :
    place = html.select_one('li:nth-child(1) > div > a').text
    place = place.split('(')[0]
    return place

In [ ]:
def get_exhibition_date(html) :
    date = html.select_one('li:nth-child(2) > div > p').text
    startAt = date.split(' ~')[0]
    endAt = date.split(' ~')[1]
    return startAt, endAt

In [ ]:
def get_ageLimit(html) :
    ageLimit = html.select_one('li:nth-child(4) > div > p').text
    return ageLimit

In [ ]:
def get_price(html) :
    price = html.select_one('li:nth-child(2) > span.price').text
    return price

In [ ]:
def get_main_img(html) :
    main_img = html.select_one('div.posterBoxTop > img').attrs['src']
    main_img_url = 'http:' + main_img
    return main_img_url

In [82]:
# (Method) 공연시간 정보 text 추출 
def get_exhibition_detail_info(html) :
  
  block_1 = html.select_one('div:nth-child(1) > div.contentDetail > p').text
  block_2 = html.select_one('div:nth-child(1) > div > ul > div').text
  
  return block_1, block_2

In [ ]:
# (Method) 공지사항 imgs 추출
def get_notice_img(html) :
  
  notice_image_list = html.select('div:nth-child(2) > div.contentDetail > img')
  images = []

  for i in notice_image_list:
    i = 'http:' + i.get('src')
    images.append(i)

  return images

In [ ]:
# (Method) 공지사항 texts 추출
def get_notice_txt(html) :

  notice_text_list = html.select_one('div:nth-child(2) > div.contentDetail').text

  return notice_text_list

In [ ]:
# (Method) 상세정보 imgs 추출
def get_exhibition_detail_info_img() :

  image_url_list = html.select('div.content.description > div.contentDetail > img') 
  images = []

  for i in image_url_list:
    i = 'http:' + i.get('src')
    images.append(i)

  return images

### 통합 메서드

In [87]:
def get_crawling_exhibition_info(url):
    
    print(f'url : {url}')
    
    resp = requests.get(url)
    
    print(f'resp : {resp}')
    
    driver.get (url)
    try:
        driver.find_element_by_xpath('//*[@id="popup-prdGuide"]/div/div[3]/div/a').click()
    except:
        pass
    
    page_text = driver.page_source
    
    # 접근한 페이지의 소스코드를 텍스트로 저장 (Str)
    page_text = driver.page_source

    # page_text를 html list로 변환 
    html = BeautifulSoup(page_text, 'html.parser')
    
    # print(f'html : \n {html}')
    
    title = html.select_one('div.summaryTop > h2').text
    
    # get title
    title = html.select_one('div.summaryTop > h2').text
    
    # get place name
    place = html.select_one('li:nth-child(1) > div > a').text
    place = place.split('(')[0]
    
    # get date (startAt, endAt)
    date = html.select_one('li:nth-child(2) > div > p').text
    startAt = date.split(' ~')[0]
    endAt = date.split(' ~')[1]
    
    # get ageLimit
    # li:nth-child(4) > div > p 
    # or li:nth-child(3) > div > p
    try:
        ageLimit = html.select_one('li:nth-child(4) > div > p').text
    except:
        ageLimit = html.select_one('li:nth-child(3) > div > p').text
    
    # get price
    try:
        price = html.select_one('li:nth-child(2) > span.price').text
    except:
        price = '가격 정보 없음'
    
    # get main_img_url
    main_img = html.select_one('div.posterBoxTop > img').attrs['src']
    main_img_url = 'http:' + main_img
    
    # get exhibition_datail_info 
    # 작성 예정
    
    # get exhibition_datail_info 
    # 작성 예정
    
    # get notice(img or text)
    # 작성 예정 (try - except 사용)

    # get detail_info(img)
    # 작성 예정

    # append
    title_list.append(title)
    place_list.append(place)
    startAt_list.append(startAt)
    endAt_list.append(endAt)
    ageLimit_list.append(ageLimit)
    price_list.append(price)
    main_img_url_list.append(main_img_url)
    

In [86]:
# title
title_list = []

# place
place_list = []

# date
startAt_list = []
endAt_list = []

# price
price_list = []

# ageLimit
ageLimit_list = []

# main_img
main_img_url_list = []

for url in exhibition_url :
    get_crawling_exhibition_info(url)
    time.sleep(3)

url : http://ticket.interpark.com/Ticket/Goods/GoodsInfo.asp?GoodsCode=22015433
resp : <Response [200]>
url : http://ticket.interpark.com/Ticket/Goods/GoodsInfo.asp?GoodsCode=22012659
resp : <Response [200]>
url : http://ticket.interpark.com/Ticket/Goods/GoodsInfo.asp?GoodsCode=23000216
resp : <Response [200]>
url : http://ticket.interpark.com/Ticket/Goods/GoodsInfo.asp?GoodsCode=22015987
resp : <Response [200]>


KeyboardInterrupt: 

In [83]:
# title
title_list

NameError: name 'title_list' is not defined

In [ ]:
# place
place_list

In [ ]:
# date
startAt_list
endAt_list

In [ ]:
# price
price_list

In [ ]:
# ageLimit
ageLimit_list

In [ ]:
# main_img
main_img_url_list

In [ ]:
data = {
        'title' : title_list,
        'place' : place_list,
        'startAt' : startAt_list,
        'endAt' : endAt_list,
        'price' : price_list,
        'ageLimit' : ageLimit_list,
        'main_img_url' : main_img_url_list
        }

df = pd.DataFrame(data)

df

In [ ]:
df.to_csv('exhibition.csv', index=False, encoding='utf-8-sig')